### Topic Modelling Demo Code

#### Things I want to do -
- Identify a package to build / train LDA model
- Use visualization to explore Documents -> Topics Distribution -> Word distribution

In [ ]:
!pip install pyLDAvis, gensim

In [3]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import pyLDAvis.gensim

# Text Preprocessing and model building
from gensim.corpora import Dictionary
import nltk
from nltk.stem import WordNetLemmatizer
import re
# Iteratively read files
import glob
import os

# For displaying images in ipython
from IPython.display import HTML, display

/Users/paragpradhan/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14.0, 8.7)
#warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

<h2>Latent Dirichlet Allocation</h2>
<h3>From Documents -- DTM -- LDA Model</h3>

Topic modeling aims to automatically summarize large collections of documents to facilitate organization and management, as well as search and recommendations. At the same time, it can enable the understanding of documents to the extent that humans can interpret the descriptions of topics

<img src="images/lda2.png" alt="lda" style="width:60%">
<img src="images/docs_to_lda.png" alt="ldaflow" style="width:100%">

### Load Data

In [5]:
# User defined function to read and store bbc data from multipe folders
def load_data(folder_names,root_path):
    fileNames = [path + '/' + 'bbc' +'/'+ folder + '/*.txt' for path,folder in zip([root_path]*len(folder_names),
                                                                               folder_names )]
    doc_list = []
    tags = folder_names
    for docs in fileNames:
        #print(docs)
        #print(type(docs))
        doc = glob.glob(docs) # glob method iterates through the all the text documents in a folder
        for text in doc:
            with open(text, encoding='latin1') as f:
                topic = docs.split('/')[-2]

                lines = f.readlines()
                heading = lines[0].strip()
                body = ' '.join([l.strip() for l in lines[1:]])
                doc_list.append([topic, heading, body])
        print("Completed loading data from folder: %s"%topic)
    
    print("Completed Loading entire text")
    
    return doc_list

In [6]:
folder_names = ['business','entertainment','politics','sport','tech']
docs = load_data(folder_names = folder_names, root_path = os.getcwd())

Completed loading data from folder: business
Completed loading data from folder: entertainment
Completed loading data from folder: politics
Completed loading data from folder: sport
Completed loading data from folder: tech
Completed Loading entire text


In [7]:
docs = pd.DataFrame(docs, columns=['Category', 'Heading', 'Article'])
print(docs.head())
print('\nShape of data is {}\n'.format(docs.shape))

   Category                            Heading  \
0  business    UK economy facing 'major risks'   
1  business  Aids and climate top Davos agenda   
2  business   Asian quake hits European shares   
3  business   India power shares jump on debut   
4  business    Lacroix label bought by US firm   

                                             Article  
0   The UK manufacturing sector will continue to ...  
1   Climate change and the fight against Aids are...  
2   Shares in Europe's leading reinsurers and tra...  
3   Shares in India's largest power producer, Nat...  
4   Luxury goods group LVMH has sold its loss-mak...  

Shape of data is (2225, 3)



### Extract Raw Corpus

In [8]:
articles = docs.Article.tolist()

In [9]:
print(type(articles))
print(articles[0:2])

<class 'list'>
[' The UK manufacturing sector will continue to face "serious challenges" over the next two years, the British Chamber of Commerce (BCC) has said.  The group\'s quarterly survey of companies found exports had picked up in the last three months of 2004 to their best levels in eight years. The rise came despite exchange rates being cited as a major concern. However, the BCC found the whole UK economy still faced "major risks" and warned that growth is set to slow. It recently forecast economic growth will slow from more than 3% in 2004 to a little below 2.5% in both 2005 and 2006.  Manufacturers\' domestic sales growth fell back slightly in the quarter, the survey of 5,196 firms found. Employment in manufacturing also fell and job expectations were at their lowest level for a year.  "Despite some positive news for the export sector, there are worrying signs for manufacturing," the BCC said. "These results reinforce our concern over the sector\'s persistent inability to sus

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()

### Preprocessing of Raw Text

In [48]:
from nltk.corpus import stopwords
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [51]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paragpradhan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = stopwords.word('english')

In [56]:
# Method to preprocess my raw data
def preprocessText(x):
    temp = x.lower()
    temp = re.sub(r'[^\w]', ' ', temp)
    temp = nltk.word_tokenize(temp)
    temp = [wordnet_lemmatizer.lemmatize(w) for w in temp]
    temp = [word for word in temp if word not in stopwords ]
    return temp

In [57]:
articles_final = [preprocessText(article) for article in articles]

In [ ]:
articles_final[0:2]

### Transformation of Preprocessed text into Vector form using Gensim

In [60]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(articles_final)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [61]:
print(dictionary)

Dictionary(3101 unique tokens: ['12', '18', '2', '2003', '2004']...)


In [62]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in articles_final]

In [63]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 3101
Number of documents: 2225


### Train LDA model using Gensim

In [22]:
dictionary

In [64]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 10
# iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
#     iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

### Model exploration: Top K words in each topic

In [65]:
# Print the Keyword in the 10 topics
pprint.pprint(model.print_topics(num_words= 20))
doc_lda = model[corpus]

[(0,
  '0.014*"game" + 0.009*"microsoft" + 0.008*"software" + 0.008*"virus" + '
  '0.008*"new" + 0.008*"firm" + 0.007*"file" + 0.007*"system" + 0.007*"site" + '
  '0.007*"program" + 0.007*"technology" + 0.007*"machine" + 0.006*"mr" + '
  '0.006*"mail" + 0.006*"people" + 0.006*"network" + 0.006*"user" + '
  '0.005*"sony" + 0.005*"computer" + 0.005*"window"'),
 (1,
  '0.015*"people" + 0.012*"service" + 0.011*"mobile" + 0.009*"phone" + '
  '0.008*"one" + 0.008*"could" + 0.007*"user" + 0.007*"technology" + '
  '0.007*"uk" + 0.007*"internet" + 0.006*"network" + 0.006*"broadband" + '
  '0.006*"net" + 0.006*"online" + 0.006*"system" + 0.006*"use" + '
  '0.006*"million" + 0.006*"new" + 0.006*"access" + 0.006*"company"'),
 (2,
  '0.017*"game" + 0.009*"film" + 0.008*"best" + 0.008*"time" + 0.007*"one" + '
  '0.006*"first" + 0.005*"world" + 0.005*"award" + 0.005*"player" + '
  '0.005*"like" + 0.005*"last" + 0.005*"two" + 0.004*"new" + 0.004*"team" + '
  '0.004*"back" + 0.004*"play" + 0.004*"win" 

### Model Visualization using PyLDAvis

In [66]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary=dictionary)
vis

PreparedData(topic_coordinates=          x     y  topics  cluster  Freq
topic                                   
3     -0.09 -0.14       1        1 43.81
2     -0.18  0.09       2        1 30.45
4      0.08  0.07       3        1 10.00
1      0.09 -0.07       4        1  8.84
0      0.09  0.05       5        1  6.89, topic_info=         Term     Freq    Total Category  logprob  loglift
2052     game 2,411.00 2,411.00  Default    30.00    30.00
219    people 1,815.00 1,815.00  Default    29.00    29.00
510    mobile   829.00   829.00  Default    28.00    28.00
212        mr 2,963.00 2,963.00  Default    27.00    27.00
100   service   861.00   861.00  Default    26.00    26.00
...       ...      ...      ...      ...      ...      ...
212        mr   169.15 2,963.61   Topic5    -5.04    -0.19
2607      dvd   116.96   556.86   Topic5    -5.41     1.11
434      make   118.62 1,067.87   Topic5    -5.39     0.48
495     could   117.28 1,488.03   Topic5    -5.40     0.13
2432    music   110.45   633.10   Topic5    -5.46     0.93

[365 rows x 6 columns], token_table=      Topic  Freq Term
term                  
123       1  0.70  000
123       2  0.06  000
123       3  0.07  000
123       4  0.10  000
123       5  0.07  000
...     ...   ...  ...
481       1  0.82    â
481       2  0.06    â
481       3  0.05    â
481       4  0.02    â
481       5  0.05    â

[935 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])

### Assign Topic Model Numbers to original Data Frame as Column

In [67]:
# Assigns the topics to the documents in corpus
lda_corpus = model[corpus]

In [68]:
topics = []

for doc in lda_corpus:
    temp_id = []
    temp_score = []
    for doc_tuple in doc:
        temp_id.append(doc_tuple[0])
        temp_score.append(doc_tuple[1])
    index = np.argmax(temp_score)
    topics.append(temp_id[index])

In [69]:
docs["Topic_num"] = topics

In [75]:
docs.tail(n= 40)

,Category,Heading,Article,Topic_num
2185,tech,Ask Jeeves joins web log market,Ask Jeeves has bought the Bloglines website t...,4
2186,tech,US duo in first spam conviction,A brother and sister in the US have been conv...,3
2187,tech,'Ultimate game' award for Doom 3,Sci-fi shooter Doom 3 has blasted away the co...,2
2188,tech,BBC leads interactive Bafta wins,The BBC and the National Theatre have led the...,2
2189,tech,Xbox 2 may be unveiled in summer,Details of the next generation of Microsoft's...,0
2190,tech,Smart search lets art fans browse,"If you don't know art but know what you like,...",4
2191,tech,Movie body targets children's PCs,The body that represents the US movie industr...,0
2192,tech,The gaming world in 2005,"If you have finished Doom 3, Half Life 2 and ...",2
2193,tech,Microsoft sets sights on spyware,Windows users could soon be paying Microsoft ...,0
2194,tech,Web photo storage market hots up,An increasing number of firms are offering we...,4
